In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import Tuple
import os
from math import ceil
import seaborn as sns

In [2]:
data_folder = 'C:/Data/Musicbrainz'

In [3]:
df = pd.read_csv(os.path.join(data_folder, 'musicbrainz-200-A01.csv'))
df

,TID,CID,CTID,SourceID,id,number,title,length,artist,album,year,language
0,1,1,1,1,WoM5452845,0,L'enfant aux yeux d'Italie (De vous à elle en ...,03:39,Daniel Balavoine,De vous à elle en passsant par moi,1975,NaN
1,2,32522,4,2,MBox36398328-HH,11,Silver Forest - 双剣乱舞 みょん＆me,229,NaN,Silver Forest 2006-2012 BEST1,12,Japanese
2,3,53749,2,3,4382873MB-01,2,shabnavard - Chavoush 2,27.183,شهرام ناظری,NaN,NaN,Persian
3,4,2,1,3,unk.,17,Mustard Gas - There and Back Again Lane,2.15,Action Painting!,NaN,'95,English
4,5,31368,3,4,215214-A048,6,006-Immer bis ich reier',3m 48sec,Nordwand,Das Pinke Album (unknown),NaN,Ger.
...,...,...,...,...,...,...,...,...,...,...,...,...
193745,193746,99998,1,4,159531-A065,2,002-Mayhem,5m 23sec,Kid Unknown,This Is Rough Mix (1992),NaN,Eng.
193746,193747,54645,2,1,WoM22726427,005,My Life (Blind Ravage),03:52,Blind Ravage,Blind Ravage,NaN,NaN
193747,193748,99997,2,5,14187740,24,"Sonata in D Minor for Cello and Piano, L 135: ...",216973,Claude Debussy,The 99 Most Essential Debussy Masterpieces,2010,NaN
193748,193749,99999,1,5,6431272,13,Make It Alone,184466,Powerplay,Ad Bouman's Goud Van Oud,1997,[Multiple languages]


Clean up the data by removing all the empy entires.

In [4]:
df = df.astype({
    'TID': 'Int64',
    'CID': 'Int64',
    'CTID': 'Int32'})

In [5]:
nan_cols = ['SourceID', 'id', 'number', 'title', 'length', 'artist', 'album', 'year', 'language', ]
for col in nan_cols:
    df = df.astype({col: 'str'})
    df[col]  = df[col].apply(lambda x : '' if x == 'nan' else x)
df.head()

,TID,CID,CTID,SourceID,id,number,title,length,artist,album,year,language
0,1,1,1,1,WoM5452845,0,L'enfant aux yeux d'Italie (De vous à elle en ...,03:39,Daniel Balavoine,De vous à elle en passsant par moi,1975,
1,2,32522,4,2,MBox36398328-HH,11,Silver Forest - 双剣乱舞 みょん＆me,229,,Silver Forest 2006-2012 BEST1,12,Japanese
2,3,53749,2,3,4382873MB-01,2,shabnavard - Chavoush 2,27.183,شهرام ناظری,,,Persian
3,4,2,1,3,unk.,17,Mustard Gas - There and Back Again Lane,2.15,Action Painting!,,'95,English
4,5,31368,3,4,215214-A048,6,006-Immer bis ich reier',3m 48sec,Nordwand,Das Pinke Album (unknown),,Ger.


## Calculate Ground Truth Stats
Before we continue let's calculate some stats that we'll need later on.

In [6]:
df_cluster_group = df.groupby(['CID'])
cluster_df = df_cluster_group.agg({'CID': 'max', 'TID': 'count'}).reset_index(drop=True)
cluster_df = cluster_df.rename(columns={'CID': 'cluster_id', 'TID': 'num_ids'})
cluster_df

,cluster_id,num_ids
0,1,1
1,2,1
2,3,1
3,4,4
4,5,1
...,...,...
99995,99996,1
99996,99997,2
99997,99998,3
99998,99999,3


Calculate the total possible number of **TID**s that can be in a cluster group. These are those cluster groups that have 2 or more **TID**s in.

In [7]:
tot_dups_df = cluster_df[cluster_df['num_ids'] > 1].agg({'num_ids': 'sum'}).reset_index()
tot_possible_dups = tot_dups_df.iloc[0,1]
print(f'total possible duplicates: {tot_possible_dups:,}')

total possible duplicates: 143,750


Calculate the total possible number of **TID**s that can't be in a cluster group, i.e. those cluster IDs that only have one **TID**.

In [8]:
tot_non_dups_df = cluster_df[cluster_df['num_ids'] == 1].agg({'num_ids': 'sum'}).reset_index()
tot_possible_non_dups = tot_non_dups_df.iloc[0,1]
print(f'total possible non-duplicates: {tot_possible_non_dups:,}')

total possible non-duplicates: 50,000


In [12]:
df_vals_full = df.groupby('CID').agg({'CID': 'max', 'TID': ['unique', 'count']})
df_vals_full

CID                        TID      
           max                     unique count
CID                                            
1            1                        [1]     1
2            2                        [4]     1
3            3                        [6]     1
4            4  [7, 51508, 54924, 117061]     4
5            5                       [10]     1
...        ...                        ...   ...
99996    99996                   [193742]     1
99997    99997           [193744, 193748]     2
99998    99998    [21780, 134402, 193746]     3
99999    99999    [52413, 167647, 193749]     3
100000  100000            [57064, 193750]     2

[100000 rows x 3 columns]

In [15]:
df_vals_full.columns = ["_".join(a) for a in df_vals_full.columns.to_flat_index()]
df_vals_full

,CID_max,TID_unique,TID_count
CID,,,
1,1,[1],1
2,2,[4],1
3,3,[6],1
4,4,"[7, 51508, 54924, 117061]",4
5,5,[10],1
...,...,...,...
99996,99996,[193742],1
99997,99997,"[193744, 193748]",2
99998,99998,"[21780, 134402, 193746]",3


In [17]:
df_cluster_groups = df_vals_full[df_vals_full['TID_count'] > 1]
df_cluster_groups

,CID_max,TID_unique,TID_count
CID,,,
4,4,"[7, 51508, 54924, 117061]",4
6,6,"[13, 90698]",2
7,7,"[14, 21850, 66635]",3
9,9,"[17, 39545, 57941, 127505]",4
10,10,"[19, 72587, 83195, 102746, 113728]",5
...,...,...,...
99995,99995,"[28434, 193740]",2
99997,99997,"[193744, 193748]",2
99998,99998,"[21780, 134402, 193746]",3


In [29]:
tid_1 = []
tid_2 = []
for i in range(len(df_cluster_groups)):
    tid_list = df_cluster_groups['TID_unique'].iloc[i]
    for j in range(0, len(tid_list)-1):
        for k in range(j+1, len(tid_list)):
            tid_1.append(f'\"{tid_list[j]}\"')
            tid_2.append(f'\"{tid_list[k]}\"')
df_gold = pd.DataFrame({'tid1_id': tid_1, 'tid2_id': tid_2})
df_gold.head(10)

,tid1_id,tid2_id
0,"""7""","""51508"""
1,"""7""","""54924"""
2,"""7""","""117061"""
3,"""51508""","""54924"""
4,"""51508""","""117061"""
5,"""54924""","""117061"""
6,"""13""","""90698"""
7,"""14""","""21850"""
8,"""14""","""66635"""
9,"""21850""","""66635"""


In [30]:
df_gold.to_csv(os.path.join(data_folder, 'musicbrainz_200k_gold_standard.csv'), index=False, sep=';')